# Data Preprocessing

## Affordability Metrics

**Dataset: Housing Costs**

In [2]:
import pandas as pd
import os

In [ ]:
# Define relative input and output paths
input_file_path = "../data/clean_data/affordability_metrics/housing_costs/metro_housing_prices.csv"
output_file_path = "../data/preprocessed_data/affordability_metrics/housing_costs/metro_housing_prices_preprocessed.csv"

In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

In [ ]:
# Step 1: Load the data
housing_prices_df = pd.read_csv(input_file_path)

In [ ]:
# Step 2: Convert data types
# Convert 'Month of Period End' to datetime
housing_prices_df['Month of Period End'] = pd.to_datetime(housing_prices_df['Month of Period End'])

# Convert numerical-like object columns to floats after removing commas
for col in ['Homes Sold', 'New Listings', 'Inventory']:
    housing_prices_df[col] = housing_prices_df[col].replace({',': ''}, regex=True).astype(float)

In [ ]:
# Step 3: Handle missing values
# Forward-fill for time-series continuity
housing_prices_df.sort_values(by=['city', 'Month of Period End'], inplace=True)
housing_prices_df.fillna(method='ffill', inplace=True)
housing_prices_df.fillna(method='bfill', inplace=True)  # Back-fill as a fallback

In [ ]:
# Step 4: Feature Engineering
# Add lagged features for 'Median Sale Price'
housing_prices_df['Median Sale Price Lag1'] = housing_prices_df.groupby('city')['Median Sale Price'].shift(1)

# Add rolling averages (3-month rolling mean for 'Median Sale Price')
housing_prices_df['Median Sale Price Rolling3'] = housing_prices_df.groupby('city')['Median Sale Price'].rolling(window=3).mean().reset_index(0, drop=True)

In [ ]:
# Step 5: Normalize Data
# Scale numerical columns (optional, depending on modeling approach)
# Skipping normalization here, but can be added later as needed

# Step 6: Save preprocessed data
housing_prices_df.to_csv(output_file_path, index=False)

**Dataset: Cost Burden**

In [ ]:
# Define relative input and output paths
input_dir = "../data/clean_data/affordability_metrics/cost_burden"
output_file_path = "../data/preprocessed_data/affordability_metrics/cost_burden/cost_burden_preprocessed.csv"

In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

In [ ]:
# Step 1: Load and merge all cost burden files into a single DataFrame
all_cost_burden_dfs = []

for file_name in os.listdir(input_dir):
    file_path = os.path.join(input_dir, file_name)
    if file_name.endswith(".csv"):
        df = pd.read_csv(file_path)
        df['source_file'] = file_name  # Add source file information for traceability
        all_cost_burden_dfs.append(df)

# Concatenate all DataFrames
merged_cost_burden_df = pd.concat(all_cost_burden_dfs, ignore_index=True)

In [ ]:
# Step 2: Transform the 'year' column into a representative midpoint year
def calculate_midpoint(year_range):
    start_year, end_year = map(int, year_range.split('-'))
    return (start_year + end_year) // 2

merged_cost_burden_df['year'] = merged_cost_burden_df['year'].apply(calculate_midpoint)

In [ ]:
# Step 3: Handle missing values
# Fill missing values using forward fill and backward fill
merged_cost_burden_df.fillna(method='ffill', inplace=True)
merged_cost_burden_df.fillna(method='bfill', inplace=True)

In [ ]:
# Step 4: Save the preprocessed data to a CSV file
merged_cost_burden_df.to_csv(output_file_path, index=False)

# Confirm the process is complete
print(f"Preprocessed cost burden data has been saved to {output_file_path}")

## Housing Supply

In [ ]:
# Define relative input and output paths
input_file_path = "../data/clean_data/housing_supply/residential_construction_permits_2000_2022.csv"
output_file_path = "../data/preprocessed_data/housing_supply/residential_construction_permits_2000_2022.csv"

In [ ]:
# Create the output directory if it doesn't exist
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

In [ ]:
# Step 1: Load the data
housing_supply_df = pd.read_csv(input_file_path)

In [ ]:
# Step 2: Reshape data from wide to long format
# Identify columns to melt (years)
id_vars = ['GEOID', 'NAME', 'STATE_NAME']
value_vars = [col for col in housing_supply_df.columns if col not in id_vars]

# Melt the dataset into long format
housing_supply_long = housing_supply_df.melt(
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='Year_Permit_Type',
    value_name='Permits'
)

In [ ]:
# Step 3: Split 'Year_Permit_Type' into 'Year' and 'Permit_Type'
housing_supply_long[['Permit_Type', 'Year']] = housing_supply_long['Year_Permit_Type'].str.extract(r'(.+)_(\d{4})')

# Convert 'Year' to an integer
housing_supply_long['Year'] = housing_supply_long['Year'].astype(int)

In [ ]:
# Step 4: Handle missing values
# Forward-fill and backward-fill missing values grouped by region
housing_supply_long.sort_values(by=['GEOID', 'Year'], inplace=True)
housing_supply_long['Permits'] = housing_supply_long.groupby('GEOID')['Permits'].transform(
    lambda group: group.fillna(method='ffill').fillna(method='bfill')
)

In [ ]:
# Step 5: Feature engineering
# Add rolling average for permits (3-year rolling mean)
housing_supply_long['Rolling_Avg_Permits'] = housing_supply_long.groupby(['GEOID', 'Permit_Type'])['Permits'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)

In [ ]:
# Step 6: Save preprocessed data
housing_supply_long.to_csv(output_file_path, index=False)

## Demand

**Dataset: Unemployment Rate**

In [ ]:
# Define file paths
input_file_path = "../data/clean_data/demand/economic/unemployment_rate/msa_unemployment.csv"
output_file_path = "../data/preprocessed_data/demand/economic/unemployment_rate/msa_unemployment.csv"

In [ ]:
# Step 1: Load the dataset
unemployment_rate_df = pd.read_csv(input_file_path)

# Step 2: Convert `DATE` column to datetime format
unemployment_rate_df['DATE'] = pd.to_datetime(unemployment_rate_df['DATE'])

In [ ]:
# Step 3: Handle missing data
# Set 'DATE' as the index for time-based interpolation
unemployment_rate_df.set_index('DATE', inplace=True)

# Use linear interpolation with respect to the index
unemployment_rate_df.interpolate(method='linear', inplace=True)

# Reset the index back to default
unemployment_rate_df.reset_index(inplace=True)

In [ ]:
# Step 4: Reshape data to long format
unemployment_rate_long = unemployment_rate_df.melt(
    id_vars=['DATE'],
    var_name='Metro_Area_Code',
    value_name='Unemployment_Rate'
)

# Step 5: Save preprocessed data
os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
unemployment_rate_long.to_csv(output_file_path, index=False)

# Display a preview of the preprocessed data
print(unemployment_rate_long.head())

**Dataset: Wages**

In [ ]:
# Define file paths
input_file_path = "../data/clean_data/demand/economic/wages/msa_wages.csv"
output_file_path = "../data/preprocessed_data/demand/economic/wages/msa_wages.csv"

In [ ]:
# Step 1: Load the dataset
wages_df = pd.read_csv(input_file_path)

In [ ]:
# Step 2: Convert `DATE` column to datetime format
wages_df['DATE'] = pd.to_datetime(wages_df['DATE'])

In [ ]:
# Step 3: Handle missing data
# Set 'DATE' as the index for time-based interpolation
wages_df.set_index('DATE', inplace=True)

# Use linear interpolation to fill missing values
wages_df.interpolate(method='linear', inplace=True)

# Reset the index back to default
wages_df.reset_index(inplace=True)

In [ ]:
# Step 4: Reshape data to long format
wages_long = wages_df.melt(
    id_vars=['DATE'],
    var_name='Metro_Area_Code',
    value_name='Average_Weekly_Wage'
)

In [ ]:
# Step 5: Save preprocessed data
os.makedirs("../data/preprocessed_data/demand/economic/unemployment_rate", exist_ok=True)
wages_long.to_csv(output_file_path, index=False)

# Display a preview of the preprocessed data
print(wages_long.head())

### Migration

In [4]:
# Define file paths
input_file_path = "../data/clean_data/demand/migration"
output_file_path = "../data/preprocessed_data/demand/migration/migration_combined.csv"

In [6]:
# Step 1: List all CSV files in the directory
migration_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(input_file_path)
    for file in files if file.endswith('.csv')
]

In [8]:
# Step 2: Load and combine all migration files
migration_dfs = []

for file_path in migration_files:
    # Extract the year from the file name
    year = os.path.basename(file_path).split('_')[-1].split('.')[0]
    
    try:
        # Load the file
        df = pd.read_csv(file_path)
        
        # Check if the file is not empty and has valid columns
        if not df.empty and 'NAME' in df.columns:
            df['Year'] = int(year)  # Add a `Year` column
            migration_dfs.append(df)
        else:
            print(f"Skipping file with issues: {file_path}")
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")

# Combine all dataframes
if migration_dfs:
    combined_migration_df = pd.concat(migration_dfs, ignore_index=True)
    print("Combined migration data loaded successfully.")
else:
    print("No valid migration files found.")

Combined migration data loaded successfully.


In [10]:
# Debugging: Print the paths and inspect a few files manually
for file_path in migration_files[:5]:  # Check the first 5 files
    try:
        # Load the file
        df = pd.read_csv(file_path)
        print(f"File: {file_path}")
        print(df.head())  # Preview the content
        print(f"Number of rows: {len(df)}")
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")

File: ../data/clean_data/demand/migration\inflow-outflow_2005.csv
                              NAME  B07201_014E  B07201_013E  B07201_012E  \
0  San Luis Obispo-Paso Robles, CA        899.0        200.0        778.0   
1    Santa Barbara-Santa Maria, CA       2725.0          0.0        166.0   
2       Santa Cruz-Watsonville, CA       2091.0        101.0        534.0   
3                     Santa Fe, NM        935.0        432.0       4543.0   
4          Santa Rosa-Petaluma, CA       1653.0        467.0        521.0   

   B07201_011E  B07201_010E  B07201_009E  B07201_008E  B07201_007E  \
0        553.0       1331.0      10085.0       6401.0      16486.0   
1        375.0        541.0      11348.0      12246.0      23594.0   
2         58.0        592.0       4430.0       5159.0       9589.0   
3        364.0       4907.0       2301.0       5354.0       7655.0   
4        882.0       1403.0       5506.0       8401.0      13907.0   

   B07201_006E  B07201_005E  B07201_004E  B07201_0

In [12]:
# Step 3: Handle missing values
# Fill missing values using forward-fill and backward-fill methods
combined_migration_df.fillna(method='ffill', inplace=True)
combined_migration_df.fillna(method='bfill', inplace=True)

C:\Users\deeps\AppData\Local\Temp\ipykernel_35696\2084664700.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_migration_df.fillna(method='ffill', inplace=True)
C:\Users\deeps\AppData\Local\Temp\ipykernel_35696\2084664700.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_migration_df.fillna(method='bfill', inplace=True)


In [14]:
# Step 4: Save the preprocessed data
os.makedirs("../data/preprocessed_data/demand/migration", exist_ok=True)
combined_migration_df.to_csv(output_file_path, index=False)

# Display a preview of the preprocessed dataset
print(combined_migration_df.head())

                              NAME  B07201_014E  B07201_013E  B07201_012E  \
0  San Luis Obispo-Paso Robles, CA        899.0        200.0        778.0   
1    Santa Barbara-Santa Maria, CA       2725.0          0.0        166.0   
2       Santa Cruz-Watsonville, CA       2091.0        101.0        534.0   
3                     Santa Fe, NM        935.0        432.0       4543.0   
4          Santa Rosa-Petaluma, CA       1653.0        467.0        521.0   

   B07201_011E  B07201_010E  B07201_009E  B07201_008E  B07201_007E  \
0        553.0       1331.0      10085.0       6401.0      16486.0   
1        375.0        541.0      11348.0      12246.0      23594.0   
2         58.0        592.0       4430.0       5159.0       9589.0   
3        364.0       4907.0       2301.0       5354.0       7655.0   
4        882.0       1403.0       5506.0       8401.0      13907.0   

   B07201_006E  B07201_005E  B07201_004E  B07201_003E  B07201_002E  \
0      21954.0      12722.0      34676.0      

### Population

**Dataset: Historical**

In [19]:
# Define file paths
input_file_path = "../data/clean_data/demand/population/historical/msa_historical.csv"
output_file_path = "../data/preprocessed_data/demand/population/historical/msa_historical.csv"

In [21]:
# Step 1: Load the dataset
historical_population_df = pd.read_csv(input_file_path)

In [23]:
# Step 2: Convert `DATE` column to datetime format
historical_population_df['DATE'] = pd.to_datetime(historical_population_df['DATE'])

In [25]:
# Step 3: Save the preprocessed data
os.makedirs("../data/preprocessed_data/demand/population/historical", exist_ok=True)
historical_population_df.to_csv(output_file_path, index=False)

In [27]:
# Display a preview of the preprocessed dataset
print(historical_population_df.head())

        DATE  POPULATION                            MSA
0 2000-01-01    4281.905  atlanta-sandy_springs-roswell
1 2001-01-01    4432.950  atlanta-sandy_springs-roswell
2 2002-01-01    4555.490  atlanta-sandy_springs-roswell
3 2003-01-01    4673.146  atlanta-sandy_springs-roswell
4 2004-01-01    4802.300  atlanta-sandy_springs-roswell


**Dataset: Estimated**

In [31]:
# Define file paths
input_file_path = "../data/clean_data/demand/population/projected/msa_projected.csv"
output_file_path = "../data/preprocessed_data/demand/population/projected/msa_projected.csv"

In [33]:
# Step 1: Load the dataset
projected_population_df = pd.read_csv(input_file_path)

In [35]:
# Step 2: Handle missing data
# Fill missing values with forward-fill and backward-fill
projected_population_df.fillna(method='ffill', inplace=True)
projected_population_df.fillna(method='bfill', inplace=True)

C:\Users\deeps\AppData\Local\Temp\ipykernel_35696\1426545730.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  projected_population_df.fillna(method='ffill', inplace=True)
C:\Users\deeps\AppData\Local\Temp\ipykernel_35696\1426545730.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  projected_population_df.fillna(method='bfill', inplace=True)


In [37]:
# Step 3: Reshape data to long format
projected_population_long = projected_population_df.melt(
    id_vars=['Geographic Area', 'Base Population (2020)'],
    var_name='Year',
    value_name='Population'
)

In [39]:
# Extract the year from the 'Year' column
projected_population_long['Year'] = projected_population_long['Year'].str.extract(r'(\d{4})').astype(int)

In [41]:
# Step 4: Save the preprocessed data
os.makedirs("../data/preprocessed_data/demand/population/projected", exist_ok=True)
projected_population_long.to_csv(output_file_path, index=False)

# Display a preview of the preprocessed dataset
print(projected_population_long.head())

               Geographic Area  Base Population (2020)  Year  Population
0                  Abilene, TX                176562.0  2020    176883.0
1                    Akron, OH                702225.0  2020    701674.0
2                   Albany, GA                148915.0  2020    148249.0
3                   Albany, OR                128611.0  2020    128981.0
4  Albany-Schenectady-Troy, NY                899247.0  2020    899724.0
